# AI-Assisted Programming
> Huggingface Transformers and OpenAI

Leveraging packages for solutions

## Class Overview
* Quick review of Gradio
* Huggingface Transformers
  * Inference
  * Tasks
* OpenAI

In [ ]:
%%capture
#!pip install transformers gradio openai diffusers accelerate

# Gradio Review

## On your own
Use generative AI (or not, but probably so) to help you create:
* a gr.Blocks() interface with
* a file upload button,
* an (input) text box
* an (output) text box,
* and a submit button.

In [ ]:
import gradio as gr

# define function that does the work
def file_process(text_in, file_in):
    return_msg = ''
    
    if file_in:
        return_msg = 'File received'
    else:
        return_msg = 'File not received'

    return return_msg

# define the user interface
with gr.Blocks() as demo:

    with gr.Row():
        with gr.Column():
            file_upload = gr.File()
            text_in = gr.Textbox(label='Query')
        text_out = gr.TextArea(label='Response')
    submit_btn = gr.Button(value='Submit', variant='primary')

    # define the connections between the block elements and the output
    submit_btn.click(file_process, inputs=[text_in, file_upload], outputs=[text_out])

# start the program
demo.launch(show_error=True)

# Huggingface Transformers
* [A walk through the possibilities](https://huggingface.co/docs/transformers/index) : tasks and models available
* [Example: Image-to-text](https://huggingface.co/Salesforce/blip-image-captioning-large)

Model recommendations:
* Companies you know
* Model card with solid description of behavior
* **READ THE MODEL CARD**
* High downloads

## Using models
Primarily: [pipelines](https://huggingface.co/docs/transformers/main_classes/pipelines)

In [ ]:
from transformers import pipeline

In [ ]:
# load document
with open('dsminor_document.txt', 'r') as f:
    doc = f.read()

In [ ]:
# use default in pytorch
summarizer = pipeline("summarization")
summarizer(doc, min_length=5, max_length=500, do_sample=False)

In [ ]:
# use models to select a model
falconzier = pipeline("summarization", model='Falconsai/text_summarization')
falconzier(doc, min_length=5, max_length=200)

Note: you can use config to get rid of the error above

### Other times, you should just use what the model card says:

In [ ]:
import torch
from diffusers import ShapEPipeline
from diffusers.utils import export_to_gif


model_name = "openai/shap-e"
pipe = ShapEPipeline.from_pretrained(model_name)

In [ ]:
guidance_scale = 15.0
prompt = "a running dog"
images = pipe(
    prompt,
    guidance_scale=guidance_scale,
    num_inference_steps=64,
    frame_size=256,
).images

In [ ]:
gif_path = export_to_gif(images, "running_dog.gif")

## Relation to gradio app
Now, let's say that we actually want to make a gradio app do this. How can we modify the app above to make this happen?

In [ ]:
import gradio as gr

In [ ]:
# use default in pytorch
summarizer = pipeline("summarization")

# define function that does the work
def file_process(text_in, file_in):
    with open(file_in, 'r') as f:
        doc = f.read()
        
    return_msg = summarizer(doc, min_length=5, max_length=500, do_sample=False)[0]['summary_text']

    return return_msg

# define the user interface
with gr.Blocks() as demo:

    with gr.Row():
        with gr.Column():
            file_upload = gr.File()
            text_in = gr.Textbox(label='Query')
        text_out = gr.TextArea(label='Response')
    submit_btn = gr.Button(value='Submit', variant='primary')

    # define the connections between the block elements and the output
    submit_btn.click(file_process, inputs=[text_in, file_upload], outputs=[text_out])

# start the program
demo.launch(show_error=True)

## What happens if you don't want to download the model?
Use [API Inference Endpoints](https://huggingface.co/docs/api-inference/index). This means that you'll need a HF token for private models.

In [ ]:
from huggingface_hub import login

In [ ]:
login()

In [ ]:
import requests

headers = {"Authorization": f"Bearer {open('/Users/bellcs1/.cache/huggingface/token').read()}"}
API_URL = "https://api-inference.huggingface.co/models/facebook/bart-large-cnn"

def query(payload):
    response = requests.post(API_URL, headers=headers, json=payload)
    return response.json()

message = {
        "inputs": doc,
        "parameters": {"do_sample": False, 'min_length':5, 'max_length':200, },
        "options": {'wait_on_model': True}
    }
    
response = query(message)

In [ ]:
response

In [ ]:
# define function that does the work
def file_process(text_in, file_in):
    with open(file_in, 'r') as f:
            doc = f.read()
    
    message['inputs'] = doc
    return_msg = query(message)[0]['generated_text']

    return return_msg

# define the user interface
with gr.Blocks() as demo:

    with gr.Row():
        with gr.Column():
            file_upload = gr.File()
            text_in = gr.Textbox(label='Query')
        text_out = gr.TextArea(label='Response')
    submit_btn = gr.Button(value='Submit', variant='primary')

    # define the connections between the block elements and the output
    submit_btn.click(file_process, inputs=[text_in, file_upload], outputs=[text_out])

# start the program
demo.launch(show_error=True)

# Introduction to OpenAI API
[OpenAI API](https://platform.openai.com/docs/overview)

Make sure to create your API key and **DO NOT** post it anywhere publicly. 100% you will be broke within hours!